In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pathlib

import numpy as np
import sklearn.externals.joblib as joblib

import pytoolkit as tk
import _data, _evaluation

In [3]:
X, y = _data.load_train_data()

In [4]:
# pred = joblib.load('cache/val/stack_dense.pkl')
pred = joblib.load('cache/val/stack_drop.pkl')
# pred = joblib.load('cache/val/stack_res.pkl')

In [13]:
mask_neg = y.max(axis=(1, 2, 3)) == 0
mask_pos = np.logical_not(mask_neg)

In [ ]:
threshold_y = []
y_pos = y > 0.5
y_neg = np.logical_not(y_pos)
for i in tk.tqdm(range(len(pred))):
    if mask_pos[i]:
        mean_p = pred[i][y_pos[i]].mean()
        mean_n = pred[i][y_neg[i]].mean()
        threshold_list = np.linspace(mean_n, mean_p, 1000)
        iou_list = []
        for th in threshold_list:
            inter = np.logical_and(pred[i] > th, y_pos[i])
            union = np.logical_or(pred[i] > th, y_pos[i])
            iou = np.sum(inter) / max(np.sum(union), 1)
            iou_list.append(iou)
        best_index = np.argmax(iou_list)
        true_theshold = threshold_list[best_index]
        iou = iou_list[best_index]
    else:
        true_theshold = (pred[i].max() + 1) / 2
    threshold_y.append(true_theshold)

pathlib.Path('threshold').mkdir(exist_ok=True)
joblib.dump(np.array(threshold_y), 'threshold/y.pkl')

In [39]:
threshold_X = np.swapaxes([
    np.min(pred, axis=(1, 2, 3)),
    np.max(pred, axis=(1, 2, 3)),
    np.mean(pred, axis=(1, 2, 3)),
    np.median(pred, axis=(1, 2, 3)),
    np.percentile(pred, 10, axis=(1, 2, 3)),
    np.percentile(pred, 20, axis=(1, 2, 3)),
    np.percentile(pred, 30, axis=(1, 2, 3)),
    np.percentile(pred, 40, axis=(1, 2, 3)),
    np.percentile(pred, 60, axis=(1, 2, 3)),
    np.percentile(pred, 70, axis=(1, 2, 3)),
    np.percentile(pred, 80, axis=(1, 2, 3)),
    np.percentile(pred, 90, axis=(1, 2, 3)),
], 0, 1)
print(threshold_X.shape)

pathlib.Path('threshold').mkdir(exist_ok=True)
joblib.dump(threshold_X, 'threshold/X.pkl')

(4000, 12)


['data/threshold_X.pkl']

In [41]:
import sklearn.ensemble
import sklearn.svm
import sklearn.neighbors
estimators = [
    sklearn.ensemble.RandomForestRegressor(n_estimators=100, n_jobs=-1),
    sklearn.ensemble.ExtraTreesRegressor(n_estimators=500, n_jobs=-1),
    sklearn.ensemble.GradientBoostingRegressor(n_estimators=100),
    sklearn.svm.SVR(gamma='auto'),
    sklearn.neighbors.KNeighborsRegressor(),
]
score_list = []
for estimator in estimators:
    pred = sklearn.model_selection.cross_val_predict(estimator, threshold_X, threshold_y, cv=5, n_jobs=-1)
    score = sklearn.metrics.mean_absolute_error(threshold_y, pred)
    score_list.append(score)
    print(f'{estimator.__class__.__name__}: {score:.3f}')

estimator = estimators[np.argmin(score_list)]
estimator.fit(threshold_X, threshold_y)
error = sklearn.metrics.mean_absolute_error(threshold_y, estimator.predict(threshold_X))
print(f'{estimator.__class__.__name__}: error={error:.3f}')
joblib.dump(estimator, 'threshold/model.pkl')

RandomForestRegressor: 0.159
ExtraTreesRegressor: 0.162
GradientBoostingRegressor: 0.160
SVR: 0.181
KNeighborsRegressor: 0.164
RandomForestRegressor: error=0.059


['threshold/model.pkl']